# 12章　モデル選択 
## レシピ12.1　全解探索による最良モデル選択 

In [ ]:
# ライブラリをロード
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# ロジスティック回帰器を作成
logistic = linear_model.LogisticRegression(max_iter=500, solver='liblinear')

# 正則化ペナルティハイパーパラメータの候補となる値の範囲を指定
penalty = ['l1', 'l2']

# 正則化強度ハイパーパラメータの候補となる値の範囲を指定
C = np.logspace(0, 4, 10)

# ハイパーパラメータ候補辞書を作成
hyperparameters = dict(C=C, penalty=penalty)

# グリッド探索器を作成
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

# グリッド探索器を訓練
best_model = gridsearch.fit(features, target)

# 最良のモデルを表示
print(best_model.best_estimator_)

In [ ]:
np.logspace(0, 4, 10)

In [ ]:
# 最良のハイパーパラメータを表示
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
# ターゲットベクトルを予測
best_model.predict(features)

## レシピ12.2　ランダム探索による最良モデルの選択 


In [ ]:
# ライブラリをロード
from scipy.stats import uniform
from sklearn import linear_model, datasets
from sklearn.model_selection import RandomizedSearchCV

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# ロジスティック回帰器を作成
logistic = linear_model.LogisticRegression(max_iter=500, solver='liblinear')

# 正則化ペナルティハイパーパラメータの候補となる値の範囲を指定
penalty = ['l1', 'l2']

# 正則化強度ハイパーパラメータの候補となる値の範囲を指定
C = uniform(loc=0, scale=4)

# ハイパーパラメータ候補辞書を作成
hyperparameters = dict(C=C, penalty=penalty)

# ランダム探索器を作成
randomizedsearch = RandomizedSearchCV(
logistic, hyperparameters, random_state=1, n_iter=100, cv=5, verbose=0,
n_jobs=-1)

# ランダム探索器を訓練
best_model = randomizedsearch.fit(features, target)

# 最良のモデルを表示
print(best_model.best_estimator_)

In [ ]:
# 0から4までの一様分布を作り、10点をサンプリングする
uniform(loc=0, scale=4).rvs(10)

In [ ]:
# 最良のハイパーパラメータを表示
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
# ターゲットベクトルを予測
best_model.predict(features)

## レシピ12.3　複数の学習アルゴリズムから最良のモデルを選択 


In [ ]:
# ライブラリをロード
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# 乱数シードを設定
np.random.seed(0)

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# パイプラインを作成
pipe = Pipeline([("classifier", RandomForestClassifier())])

# 候補学習アルゴリズムとそのハイパーパラメータの辞書を作成
search_space = [{"classifier": [LogisticRegression(max_iter=500, solver='liblinear')],
                 "classifier__penalty": ['l1', 'l2'],
                 "classifier__C": np.logspace(0, 4, 10)},
                 {"classifier": [RandomForestClassifier()],
                  "classifier__n_estimators": [10, 100, 1000],
                  "classifier__max_features": [1, 2, 3]}]

# グリッド探索器を作成
gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=0)

# グリッド探索器を訓練
best_model = gridsearch.fit(features, target)

# 最良のモデルを表示
print(best_model.best_estimator_)

In [ ]:
# 最良のモデルを表示
print(best_model.best_estimator_.get_params()["classifier"])

In [ ]:
# ターゲットベクトルを予測
best_model.predict(features)

## レシピ12.4　前処理がある場合の最良モデルの選択 


In [ ]:
# ライブラリをロード
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 乱数シードを設定
np.random.seed(0)

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# StandardScalerとPCAを含む前処理オブジェクトを作成
preprocess = FeatureUnion([("std", StandardScaler()), ("pca", PCA())])

# パイプラインを作成
pipe = Pipeline([("preprocess", preprocess),
                 ("classifier", LogisticRegression(max_iter=1000,
                                                   solver='liblinear'))])

# 候補値の空間を作成
search_space = [{"preprocess__pca__n_components": [1, 2, 3],
                 "classifier__penalty": ["l1", "l2"],
                 "classifier__C": np.logspace(0, 4, 10)}]

# グリッド探索器を作成
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)

# グリッド探索器を訓練
best_model = clf.fit(features, target)

# 最良のモデルを表示
print(best_model.best_estimator_)

In [ ]:
# 最良の主成分の数を表示
best_model.best_estimator_.get_params()['preprocess__pca__n_components']

## レシピ12.5　並列化によるモデル選択の高速化 


In [ ]:
# ライブラリをロード
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# ロジスティック回帰器を作成
logistic = linear_model.LogisticRegression(max_iter=500, solver='liblinear')

# 正則化ペナルティハイパーパラメータの候補となる値の範囲を指定
penalty = ["l1", "l2"]

# 正則化強度ハイパーパラメータの候補となる値の範囲を指定
C = np.logspace(0, 4, 1000)

# ハイパーパラメータ候補辞書を作成
hyperparameters = dict(C=C, penalty=penalty)

# グリッド探索器を作成
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=-1, verbose=1)

# グリッド探索器を訓練
best_model = gridsearch.fit(features, target)

# 最良のモデルを表示
print(best_model.best_estimator_)

In [ ]:
# 1コアだけ使用するグリッド探索器を作成
clf = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=1, verbose=1)

# グリッド探索器を訓練
best_model = clf.fit(features, target)

# 最良のモデルを表示
print(best_model.best_estimator_)

## レシピ12.6　アルゴリズム固有の方法を用いたモデル選択の高速化 


In [ ]:
# ライブラリをロード
from sklearn import linear_model, datasets

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# ロジスティック回帰交差検証器を作成
logit = linear_model.LogisticRegressionCV(Cs=100, max_iter=500, 
                                          solver='liblinear')

# 交差検証器を訓練
logit.fit(features, target)

# モデルを表示
print(logit)

## レシピ12.7　モデル選択後の性能評価 


In [ ]:
# ライブラリをロード
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV, cross_val_score

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# ロジスティック回帰器を作成
logistic = linear_model.LogisticRegression(max_iter=500, solver='liblinear')

# 正則化強度ハイパーパラメータの候補値を20個作成
C = np.logspace(0, 4, 20)

# ハイパーパラメータ候補辞書を作成
hyperparameters = dict(C=C)

# グリッド探索器を作成
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=-1, verbose=0)

# 二重交差検証を行い、平均値を表示
cross_val_score(gridsearch, features, target).mean()

In [ ]:
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=1)
best_model = gridsearch.fit(features, target)

In [ ]:
scores = cross_val_score(gridsearch, features, target)